In [1]:
import os
# Find the latest version of spark 2.0  from http://www-us.apache.org/dist/spark/ and enter as the spark version
# For example:
# spark_version = 'spark-3.0.0'
spark_version = 'spark-3.0.1'
os.environ['SPARK_VERSION']=spark_version

# Install Spark and Java
!apt-get update
!apt-get install openjdk-11-jdk-headless -qq > /dev/null
!wget -q http://www-us.apache.org/dist/spark/$SPARK_VERSION/$SPARK_VERSION-bin-hadoop2.7.tgz
!tar xf $SPARK_VERSION-bin-hadoop2.7.tgz
!pip install -q findspark

# Set Environment Variables
import os
os.environ["JAVA_HOME"] = "/usr/lib/jvm/java-11-openjdk-amd64"
os.environ["SPARK_HOME"] = f"/content/{spark_version}-bin-hadoop2.7"

# Start a SparkSession
import findspark
findspark.init()

Ign:1 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu1804/x86_64  InRelease
Ign:2 https://developer.download.nvidia.com/compute/machine-learning/repos/ubuntu1804/x86_64  InRelease
Hit:3 https://cloud.r-project.org/bin/linux/ubuntu bionic-cran40/ InRelease
Hit:4 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu1804/x86_64  Release
Hit:5 https://developer.download.nvidia.com/compute/machine-learning/repos/ubuntu1804/x86_64  Release
Hit:6 http://archive.ubuntu.com/ubuntu bionic InRelease
Get:7 http://security.ubuntu.com/ubuntu bionic-security InRelease [88.7 kB]
Get:9 http://archive.ubuntu.com/ubuntu bionic-updates InRelease [88.7 kB]
Hit:10 http://ppa.launchpad.net/c2d4u.team/c2d4u4.0+/ubuntu bionic InRelease
Hit:12 http://ppa.launchpad.net/cran/libgit2/ubuntu bionic InRelease
Get:13 http://archive.ubuntu.com/ubuntu bionic-backports InRelease [74.6 kB]
Hit:14 http://ppa.launchpad.net/graphics-drivers/ppa/ubuntu bionic InRelease
Fetched 252 kB in 2s (122 kB/s

In [2]:
# Download the Postgres driver that will allow Spark to interact with Postgres.
!wget https://jdbc.postgresql.org/download/postgresql-42.2.16.jar

--2021-02-08 04:07:54--  https://jdbc.postgresql.org/download/postgresql-42.2.16.jar
Resolving jdbc.postgresql.org (jdbc.postgresql.org)... 72.32.157.228, 2001:4800:3e1:1::228
Connecting to jdbc.postgresql.org (jdbc.postgresql.org)|72.32.157.228|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 1002883 (979K) [application/java-archive]
Saving to: ‘postgresql-42.2.16.jar.6’

postgresql-42.2.16. 100%[===================>] 979.38K  6.05MB/s    in 0.2s    

2021-02-08 04:07:54 (6.05 MB/s) - ‘postgresql-42.2.16.jar.6’ saved [1002883/1002883]



In [3]:
from pyspark.sql import SparkSession
spark = SparkSession.builder.appName("BigData-Challenge").config("spark.driver.extraClassPath","/content/postgresql-42.2.16.jar").getOrCreate()

In [4]:
# Configure settings for RDS
jdbc_url="jdbc:postgresql://database-challenge.cepct6ijggwk.us-east-2.rds.amazonaws.com:5432/postgres"
config = {"user":"challenge", 
          "password": "Wjp3w8PGEkFSRbJ", 
          "driver":"org.postgresql.Driver"}

In [5]:
df = spark.read.jdbc(url=jdbc_url,table="vine_table",properties=config)

In [6]:
total_votes_df = df.filter(df["total_votes"] >= 20)
total_votes_df.show()

+--------------+-----------+-------------+-----------+----+-----------------+
|     review_id|star_rating|helpful_votes|total_votes|vine|verified_purchase|
+--------------+-----------+-------------+-----------+----+-----------------+
|R1J45RVPTZ5CWW|          5|           21|         21|   N|                N|
| RZ3T3SRWVUV87|          1|           21|         24|   N|                Y|
|R23PUAT9JE3TXZ|          1|           20|         26|   N|                N|
| R8BCNNNW6UB02|          5|           32|         35|   N|                Y|
|R3A8NFYXYI0AQ3|          5|           31|         32|   N|                Y|
|R2Y3QGHTAUX7Y4|          1|           49|         70|   N|                N|
| RDT7VRCPUTFVW|          5|           19|         21|   N|                Y|
|R28M04PY6FH0Y7|          1|           23|         26|   N|                Y|
|R3HVN1S3SKR0IN|          1|           24|         29|   N|                N|
| R2K9KYE3FG84B|          2|           14|         20|   N|     

In [7]:
helpful_votes_df = total_votes_df.filter((total_votes_df["helpful_votes"]/total_votes_df["total_votes"]) >= 0.5)
helpful_votes_df.show()

+--------------+-----------+-------------+-----------+----+-----------------+
|     review_id|star_rating|helpful_votes|total_votes|vine|verified_purchase|
+--------------+-----------+-------------+-----------+----+-----------------+
|R1J45RVPTZ5CWW|          5|           21|         21|   N|                N|
| RZ3T3SRWVUV87|          1|           21|         24|   N|                Y|
|R23PUAT9JE3TXZ|          1|           20|         26|   N|                N|
| R8BCNNNW6UB02|          5|           32|         35|   N|                Y|
|R3A8NFYXYI0AQ3|          5|           31|         32|   N|                Y|
|R2Y3QGHTAUX7Y4|          1|           49|         70|   N|                N|
| RDT7VRCPUTFVW|          5|           19|         21|   N|                Y|
|R28M04PY6FH0Y7|          1|           23|         26|   N|                Y|
|R3HVN1S3SKR0IN|          1|           24|         29|   N|                N|
| R2K9KYE3FG84B|          2|           14|         20|   N|     

In [8]:
paid_reviews_df = helpful_votes_df.filter(df["vine"].isin("Y"))
paid_reviews_df.show()

+--------------+-----------+-------------+-----------+----+-----------------+
|     review_id|star_rating|helpful_votes|total_votes|vine|verified_purchase|
+--------------+-----------+-------------+-----------+----+-----------------+
|R191XSIF5H9SDP|          3|           75|         76|   Y|                N|
| RUD4ZGLPRHXK8|          5|           99|        101|   Y|                N|
|R27D5SF041PPF2|          2|           17|         20|   Y|                N|
|R18E5B8UUGTXSL|          2|           58|         69|   Y|                N|
|R213PJITYQAMNF|          3|           27|         29|   Y|                N|
|R2YT3R9SJGIBN4|          5|          424|        455|   Y|                N|
|R1XAOKR8PXM8JC|          5|           58|         59|   Y|                N|
| RTYIMPVGTA5AH|          5|           45|         50|   Y|                N|
|R32BLSG18S1GOO|          5|           40|         57|   Y|                N|
| RDVN535NACYMI|          3|           21|         22|   Y|     

In [9]:
unpaid_reviews_df = helpful_votes_df.filter(df["vine"].isin("N"))
unpaid_reviews_df.show()

+--------------+-----------+-------------+-----------+----+-----------------+
|     review_id|star_rating|helpful_votes|total_votes|vine|verified_purchase|
+--------------+-----------+-------------+-----------+----+-----------------+
|R1J45RVPTZ5CWW|          5|           21|         21|   N|                N|
| RZ3T3SRWVUV87|          1|           21|         24|   N|                Y|
|R23PUAT9JE3TXZ|          1|           20|         26|   N|                N|
| R8BCNNNW6UB02|          5|           32|         35|   N|                Y|
|R3A8NFYXYI0AQ3|          5|           31|         32|   N|                Y|
|R2Y3QGHTAUX7Y4|          1|           49|         70|   N|                N|
| RDT7VRCPUTFVW|          5|           19|         21|   N|                Y|
|R28M04PY6FH0Y7|          1|           23|         26|   N|                Y|
|R3HVN1S3SKR0IN|          1|           24|         29|   N|                N|
| R2K9KYE3FG84B|          2|           14|         20|   N|     

In [10]:
total_reviews = df.count()

In [30]:
five_star_reviews = helpful_votes_df.filter(helpful_votes_df["star_rating"] == 5).count()

In [14]:
five_star_paid_reviews = paid_reviews_df.filter(paid_reviews_df["star_rating"] == 5).count()

In [15]:
five_star_unpaid_reviews = unpaid_reviews_df.filter(unpaid_reviews_df["star_rating"] == 5).count()

In [35]:
paid_percentage = (five_star_reviews/five_star_paid_reviews)

In [36]:
unpaid_percentage = (five_star_reviews/five_star_unpaid_reviews)

In [37]:
print(paid_percentage)

70.40277777777777


In [38]:
print(unpaid_percentage)

1.0144086451871122
